In [17]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

In [18]:
df_test = pd.read_csv('time_series_data_AL.csv')

numerical_cols = ["sin_month", "cos_month", "Year", "Month", "num_days", "holiday_num"]
numerical_features = [
    "crime_count",
    "crime_pct_change",
    "morning",
    "afternoon",
    "evening",
    "night"
]

for i in [1, 2, 3, 6, 12]:
    for j in numerical_features:
        numerical_cols.append(f"{j}_lag{i}")
for i in [3, 6]:
    for j in numerical_features:
        numerical_cols.append(f"{j}_ma{i}")

X_test = df_test[numerical_cols]
y_test = df_test["crime_count"]

cat_pipe = Pipeline([
    ("encoder", OneHotEncoder(handle_unknown="ignore")),
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_cols)
    ]
)

In [19]:
X_test_processed = preprocessor.fit_transform(X_test)
X_test_tensor = torch.tensor(X_test_processed, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [20]:
class CrimeDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

test_dataset = CrimeDataset(X_test_tensor, y_test_tensor)

batch_size = 8
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [21]:
class CrimePredictionModel(nn.Module):
    def __init__(self, input_dim):
        super(CrimePredictionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

input_dim = X_test_processed.shape[1]
best_model = CrimePredictionModel(input_dim)
best_model.load_state_dict(torch.load("best_model.pth"))
best_model.eval()

CrimePredictionModel(
  (fc1): Linear(in_features=48, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [22]:
with torch.no_grad():
    y_test_pred = best_model(X_test_tensor).numpy().squeeze()

df_results = df_test.copy()
df_results['cnn_pred'] = 0.0
df_results.loc[df_test.index, 'cnn_pred'] = y_test_pred
df_results.to_csv('cnn_predictions.csv', index=False)
print("Predictions saved to 'cnn_predictions.csv'")


Predictions saved to 'cnn_predictions.csv'
